In [1]:
import os
import difflib
import zipfile
import lxml.etree as etree
import random
import re
import sys
from datetime import datetime
import pandas as pd

import django
import platform

sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import cities.models as cmodels
from django.contrib.auth.models import User
import tmv_app.models as tm

#from parsing_utils import POI, dehyphenate_with_space, clean_text, find_person_in_db_aph
#from aph_model_builder import *

/home/leey/venv/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


In [2]:
data_dir = '/home/leey/Documents/Data/AustralianPoliticians/data-raw/'

In [3]:
df_all = pd.read_csv(data_dir+'all.csv')

In [6]:
df_all2 = pd.read_csv('aphid_parliamentarians.csv')

In [4]:
df_all['deathDate'] = pd.to_datetime(df_all['deathDate'],infer_datetime_format=True)
df_all['birthDate'] = pd.to_datetime(df_all['birthDate'],infer_datetime_format=True)

In [6]:
df_all.head()

,uniqueID,surname,allOtherNames,firstName,commonName,displayName,earlierOrLaterNames,title,gender,birthDate,birthYear,deathDate,member,senator,wikidataID,wikipedia,adb,comments
0,Abbott1859,Abbott,Richard Hartley Smith,Richard,NaN,"Abbott, Richard",NaN,NaN,male,NaT,1859.0,1940-02-28,0,1,Q7323613,https://en.wikipedia.org/wiki/Richard_Abbott_(...,NaN,NaN
1,Abbott1869,Abbott,Percy Phipps,Percy,NaN,"Abbott, Percy",NaN,NaN,male,1869-05-14,NaN,1940-09-09,1,1,Q7167212,https://en.wikipedia.org/wiki/Percy_Abbott_(Au...,NaN,NaN
2,Abbott1877,Abbott,Macartney,Macartney,Mac,"Abbott, Mac",NaN,NaN,male,1877-07-03,NaN,1960-12-30,0,1,Q6722175,https://en.wikipedia.org/wiki/Mac_Abbott,NaN,NaN
3,Abbott1886,Abbott,Charles Lydiard Aubrey,Charles,Aubrey,"Abbott, Aubrey",NaN,NaN,male,1886-01-04,NaN,1975-04-30,1,0,Q4819077,https://en.wikipedia.org/wiki/Aubrey_Abbott,NaN,NaN
4,Abbott1891,Abbott,Joseph Palmer,Joseph,NaN,"Abbott, Joseph",NaN,NaN,male,1891-10-18,NaN,1965-05-07,1,0,Q6286049,https://en.wikipedia.org/wiki/Joseph_Palmer_Ab...,NaN,NaN


In [8]:
df_all2['deathDate'] = pd.to_datetime(df_all2['deathDate'],infer_datetime_format=True)
df_all2['birthDate'] = pd.to_datetime(df_all2['birthDate'],infer_datetime_format=True)

In [9]:
df_all2.head()

,uniqueID,surname,allOtherNames,firstName,commonName,displayName,earlierOrLaterNames,title,gender,birthDate,...,deathDate,member,senator,wasPrimeMinister,wikidataID,wikipedia,adb,comments,aphID,idsComments
0,Abbott1859,Abbott,Richard Hartley Smith,Richard,NaN,"Abbott, Richard",NaN,NaN,male,NaT,...,1940-02-28,0,1,NaN,Q7323613,https://en.wikipedia.org/wiki/Richard_Abbott_(...,NaN,NaN,JYU,NaN
1,Abbott1869,Abbott,Percy Phipps,Percy,NaN,"Abbott, Percy",NaN,NaN,male,1869-05-14,...,1940-09-09,1,1,NaN,Q7167212,https://en.wikipedia.org/wiki/Percy_Abbott_(Au...,NaN,NaN,JLM,NaN
2,Abbott1877,Abbott,Macartney,Macartney,Mac,"Abbott, Mac",NaN,NaN,male,1877-07-03,...,1960-12-30,0,1,NaN,Q6722175,https://en.wikipedia.org/wiki/Mac_Abbott,NaN,NaN,JNT,NaN
3,Abbott1886,Abbott,Charles Lydiard Aubrey,Charles,Aubrey,"Abbott, Aubrey",NaN,NaN,male,1886-01-04,...,1975-04-30,1,0,NaN,Q4819077,https://en.wikipedia.org/wiki/Aubrey_Abbott,NaN,NaN,JLJ,NaN
4,Abbott1891,Abbott,Joseph Palmer,Joseph,NaN,"Abbott, Joseph",NaN,NaN,male,1891-10-18,...,1965-05-07,1,0,NaN,Q6208273,https://en.wikipedia.org/wiki/Joe_Abbott_(Aust...,NaN,NaN,JLL,NaN


In [14]:
pd.isnull(df_all2['title'][0])

True

In [7]:
df_all.dtypes

uniqueID                       object
surname                        object
allOtherNames                  object
firstName                      object
commonName                     object
displayName                    object
earlierOrLaterNames            object
title                          object
gender                         object
birthDate              datetime64[ns]
birthYear                     float64
deathDate              datetime64[ns]
member                          int64
senator                         int64
wikidataID                     object
wikipedia                      object
adb                            object
comments                       object
dtype: object

### Political Party

In [22]:
df_party = pd.read_csv(data_dir+'by_party.csv')
len(df_party)

2254

In [24]:
df_party.dtypes

uniqueID                      object
partyAbbrev                   object
partyName                     object
partyFrom                     object
partyTo                       object
partyChangedName             float64
partySimplifiedName           object
partySpecificDateInputted    float64
partyComments                 object
dtype: object

In [7]:
df_party.head()

,uniqueID,partyAbbrev,partyName,partyFrom,partyTo,partyChangedName,partySimplifiedName,partySpecificDateInputted,partyComments
0,Abbott1859,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN
1,Abbott1869,CP,Australian Country Party,1919-11-03,NaN,NaN,Nationals,NaN,NaN
2,Abbott1869,LIB,Liberal Party of Australia,NaN,1917-02-17,NaN,Liberals,NaN,NaN
3,Abbott1869,NAT,Nationalist Party,1917-02-17,1919-11-03,NaN,Nationalist Party,NaN,NaN
4,Abbott1877,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN


In [20]:
df_party

,uniqueID,partyAbbrev,partyName,partyFrom,partyTo,partyChangedName,partySimplifiedName,partySpecificDateInputted,partyComments
0,Abbott1859,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN
1,Abbott1869,CP,Australian Country Party,1919-11-03,NaN,NaN,Nationals,NaN,NaN
2,Abbott1869,LIB,Liberal Party of Australia,NaN,1917-02-17,NaN,Liberals,NaN,NaN
3,Abbott1869,NAT,Nationalist Party,1917-02-17,1919-11-03,NaN,Nationalist Party,NaN,NaN
4,Abbott1877,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN
5,Abbott1886,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN
6,Abbott1891,CP,Australian Country Party,NaN,NaN,NaN,Nationals,NaN,NaN
7,Abbott1957,LIB,Liberal Party of Australia,NaN,NaN,NaN,Liberals,NaN,NaN
8,Abel1939,LIB,Liberal Party of Australia,NaN,NaN,NaN,Liberals,NaN,NaN
9,Abetz1958,LIB,Liberal Party of Australia,NaN,NaN,NaN,Liberals,NaN,NaN


In [12]:
for i in df_all.index:
    pol = df_all.loc[i]
    df_party.where(df_party['uniqueID']==pol['uniqueID']).notna()

### building a parser

In [10]:
#POL_FNAME = data_dir + 'all.csv'
POL_FNAME = 'aphid_parliamentarians.csv'

In [15]:
def parse_aph_data(verbosity=0):

    warn = 0

    df_pol = pd.read_csv(POL_FNAME)
    print("read data from {}".format(POL_FNAME))

    # going through entries for mdbs
    for i in df_pol.index:
        pol = df_pol.loc[i]

        person = pm.Person(
            surname=str(pol['surname']),
            first_name=str(pol['firstName']),
            unique_id=str(pol['uniqueID']),
            aph_id=str(pol['aphID'])
            )

        # profile information
        title = pol['title']
        if pd.isnull(title) == False:
            person.title = str(title)

        dob = pol['birthDate']
        dod = pol['deathDate']
        
        if pd.isnull(pol['birthYear']) == False:
            yob = int(pol['birthYear'])
        
        if pd.isnull(dob) == False:
            person.date_of_birth = datetime.strptime(str(dob), '%Y-%m-%d') 
        else:
            person.year_of_birth = yob
        
        if pd.isnull(dod) == False:
            person.date_of_death = datetime.strptime(str(dod), '%Y-%m-%d')

        if pol['gender'] == "male":
            person.gender = pm.Person.MALE
        elif pol['gender'] == "female":
            person.gender = pm.Person.FEMALE

        # additional names
        firstnames_list = []

        firstname = pol['allOtherNames']

        if isinstance(firstname, str):
            firstnames_list.append(firstname)

            if len(firstname.split(' ')) > 1:
                for fname in firstname.split(" "):
                    if len(fname) > 2:
                        firstnames_list.append(fname)


        commonname = pol['commonName']

        if isinstance(commonname, str):
            firstnames_list.append(commonname)

            if len(commonname.split(' ')) > 1:
                for cname in commonname.split(" "):
                    if len(cname) > 2:
                        firstnames_list.append(cname)

        person.alt_first_names = list(set(firstnames_list))

        if verbosity > 0:
            # print name
            print("Person: {}".format(person))

        person.active_country = cmodels.Country.objects.get(name='Australia')
        # adding positions
        if pol['member'] == 1:
            person.positions = ['Member']
        if pol['senator'] == 1:
            person.positions = ['Senator']

        person.information_source = "AustralianPoliticians"
        person.save()

        #person.in_parlperiod = wp_list
        #person.save()

    print("Done. {} warnings.".format(warn))

In [16]:
if __name__ == '__main__':
    delete_all = False
    delete_persons = False

    if delete_all:
        # to delete all existing entries for constituencies, partylists, seats and persons
        pm.Constituency.objects.all().delete()
        pm.PartyList.objects.all().delete()
        pm.Party.objects.all().delete()
        pm.Seat.objects.all().delete()
        pm.Person.objects.all().delete()

    elif delete_persons:
        pm.Person.objects.filter(information_source="AustralianPoliticians").delete()

    # updating the parties
    # update_parties()
    # parsing the data
    parse_aph_data(verbosity=0)

read data from aphid_parliamentarians.csv
Done. 0 warnings.


### adding parties of politicians

In [3]:
PAR_FNAME = data_dir + 'by_party.csv'

In [25]:
def add_parties(file, verbosity=1):
    df_par = pd.read_csv(PAR_FNAME)
    print("read data from {}".format(PAR_FNAME))
    
    parliament = pm.Parl.objects.get(country = cmodels.Country.objects.get(name='Australia'))
    
    for i in df_par.index:
        pol = df_par.loc[i]
        
        unique_id = str(pol['uniqueID'])
        person = pm.Person.objects.get(unique_id=unique_id)
        
        if verbosity > 0:
            print("Adding party for {}".format(unique_id))
        
        # party and party membership info 
        party_abbrev = str(pol['partyAbbrev'])
        party_name = str(pol['partyName'])
        party_simplified = str(pol['partySimplifiedName'])
        
        party_alt_names = [party_abbrev, party_simplified]
        
        # get Party object
        par_party, created = pm.Party.objects.get_or_create(
            name=party_name,
            alt_names = party_alt_names,
            parliament = parliament
        )
        #par_party.save()
                
        party_from = pol['partyFrom']
        party_to = pol['partyTo']
        
        # create PartyMembership association
        par_mem = pm.PartyMembership(
            person=person,
            party=par_party
        )
        
        if pd.isnull(party_from) == False or pd.isnull(party_to) == False:
            par_mem.period_unknown = False
            
            if pd.isnull(party_from) == False:
                par_mem.entry_date = datetime.strptime(str(party_from), '%Y-%m-%d')
            if pd.isnull(party_to) == False:
                par_mem.resignation_date = datetime.strptime(str(party_to), '%Y-%m-%d') 
            
        par_mem.save()
        
    print("Done.")

In [41]:
add_parties(file=PAR_FNAME)

read data from /home/leey/Documents/Data/AustralianPoliticians/data-raw/by_party.csv
Adding party for Abbott1859
Adding party for Abbott1869
Adding party for Abbott1869
Adding party for Abbott1869
Adding party for Abbott1877
Adding party for Abbott1886
Adding party for Abbott1891
Adding party for Abbott1957
Adding party for Abel1939
Adding party for Abetz1958
Adding party for Adams1943
Adding party for Adams1951
Adding party for Adamson1857
Adding party for Adermann1896
Adding party for Adermann1927
Adding party for Adermann1927
Adding party for Adermann1927
Adding party for Ahern1871
Adding party for Albanese1963
Adding party for Aldred1945
Adding party for Alexander1951
Adding party for Allan1916
Adding party for Allen1966
Adding party for Allison1946
Adding party for Alston1941
Adding party for Aly1967
Adding party for Amour1900
Adding party for Amour1900
Adding party for Amour1900
Adding party for Anderson1909
Adding party for Anderson1956
Adding party for Anderson1956
Adding party

Adding party for Byrne1962
Adding party for Cadman1937
Adding party for Cairns1914
Adding party for Cairns1929
Adding party for Calder1916
Adding party for Calder1916
Adding party for Calvert1940
Adding party for Calwell1896
Adding party for Cameron1851
Adding party for Cameron1857
Adding party for Cameron1873
Adding party for Cameron1873
Adding party for Cameron1873
Adding party for Cameron1878
Adding party for Cameron1879
Adding party for Cameron1895
Adding party for Cameron1895
Adding party for Cameron1895
Adding party for Cameron1895
Adding party for Cameron1900
Adding party for Cameron1913
Adding party for Cameron1914
Adding party for Cameron1917
Adding party for Cameron1930
Adding party for Cameron1935
Adding party for Cameron1938
Adding party for Cameron1938
Adding party for Cameron1940
Adding party for Cameron1965
Adding party for CameronDouglas1951
Adding party for CameronEoin1951
Adding party for Campbell1939
Adding party for Campbell1939
Adding party for Campbell1943
Adding 

Adding party for Drury1912
Adding party for Dubois1940
Adding party for Duffy1938
Adding party for Duncan1883
Adding party for Duncan1945
Adding party for DuncanHughes1882
Adding party for DuncanHughes1882
Adding party for DuncanHughes1882
Adding party for Duniam1982
Adding party for Dunn1887
Adding party for Dunn1887
Adding party for Dunn1948
Adding party for Dunn1948
Adding party for Durack1926
Adding party for Duthie1912
Adding party for Dutton1970
Adding party for Earle1865
Adding party for Easson1955
Adding party for Edmonds1903
Adding party for Edwards1842
Adding party for Edwards1842
Adding party for Edwards1842
Adding party for Edwards1842
Adding party for Edwards1855
Adding party for Edwards1855
Adding party for Edwards1927
Adding party for Edwards1945
Adding party for Edwards1946
Adding party for Edwards1962
Adding party for Eggins1898
Adding party for Eggleston1941
Adding party for Einfeld1909
Adding party for Eldridge1872
Adding party for Eldridge1872
Adding party for Ellic

Adding party for Groom1852
Adding party for Groom1867
Adding party for Groom1867
Adding party for Groom1867
Adding party for Groom1867
Adding party for Groom1867
Adding party for Groom1944
Adding party for Guilfoyle1926
Adding party for Gullett1878
Adding party for Gullett1878
Adding party for Gullett1914
Adding party for Gun1936
Adding party for Guthrie1856
Adding party for Guthrie1856
Adding party for Guthrie1872
Adding party for Guthrie1872
Adding party for Guy1860
Adding party for Guy1890
Adding party for Guy1890
Adding party for Guy1890
Adding party for Haase1945
Adding party for Hadley1893
Adding party for Haines1945
Adding party for Haines1961
Adding party for Halbert1910
Adding party for Hale1969
Adding party for Hall1874
Adding party for Hall1928
Adding party for Hall1928
Adding party for Hall1949
Adding party for Hallett1917
Adding party for Halverson1937
Adding party for Hamer1923
Adding party for Hamilton1899
Adding party for Hammond1968
Adding party for Hammond1975
Adding 

Adding party for King1973
Adding party for Kingsmill1864
Adding party for Kingston1850
Adding party for Kirk1967
Adding party for Kirwan1869
Adding party for Kirwan1937
Adding party for Kitching1970
Adding party for Klugman1924
Adding party for Kneebone1876
Adding party for Knight1943
Adding party for Knott1956
Adding party for Knowles1951
Adding party for Knox1850
Adding party for Knox1850
Adding party for Knox1850
Adding party for Kroger1959
Adding party for Lacey1887
Adding party for Lacey1900
Adding party for Lajovic1921
Adding party for Lamb1939
Adding party for Lamb1972
Adding party for Lambert1881
Adding party for Lambie1971
Adding party for Lambie1971
Adding party for Lambie1971
Adding party for Laming1966
Adding party for Lamond1865
Adding party for Lamp1895
Adding party for Landry1962
Adding party for Lane1873
Adding party for Lang1876
Adding party for Lang1876
Adding party for Langmore1939
Adding party for Langtry1880
Adding party for Large1878
Adding party for Latham1877
Ad

Adding party for McLeay1945
Adding party for McLeod1892
Adding party for McLucas1958
Adding party for McMahon1908
Adding party for McMahon1930
Adding party for McMahon1967
Adding party for McManus1905
Adding party for McManus1905
Adding party for McMillan1850
Adding party for McMullan1947
Adding party for McMullin1900
Adding party for McNamara1964
Adding party for McNeill1868
Adding party for McNeill1921
Adding party for McNicoll1877
Adding party for McTiernan1892
Adding party for McVeigh1930
Adding party for McVeigh1930
Adding party for McVeigh1930
Adding party for McVeigh1965
Adding party for McWilliams1856
Adding party for McWilliams1856
Adding party for McWilliams1856
Adding party for McWilliams1856
Adding party for McWilliams1856
Adding party for McWilliams1856
Adding party for Melham1954
Adding party for Melzer1926
Adding party for Menzies1894
Adding party for Menzies1894
Adding party for Messner1939
Adding party for Mildren1932
Adding party for Miles1947
Adding party for Millar1

Adding party for Rennick1970
Adding party for Reynolds1923
Adding party for Reynolds1941
Adding party for Reynolds1965
Adding party for Rhiannon1951
Adding party for Rice1960
Adding party for Richardson1939
Adding party for Richardson1939
Adding party for Richardson1949
Adding party for Richardson1958
Adding party for Ridgeway1962
Adding party for Ridley1909
Adding party for Riggall1941
Adding party for Riley1859
Adding party for Riley1892
Adding party for Riley1892
Adding party for Riordan1886
Adding party for Riordan1886
Adding party for Riordan1908
Adding party for Riordan1908
Adding party for Riordan1930
Adding party for Ripoll1966
Adding party for Rishworth1978
Adding party for Robb1951
Adding party for Robert1970
Adding party for Roberton1900
Adding party for Roberts1869
Adding party for Roberts1955
Adding party for Robertson1882
Adding party for Robertson1882
Adding party for Robertson1929
Adding party for Robinson1872
Adding party for Robinson1877
Adding party for Robinson1907


Adding party for Thomson1955
Adding party for Thomson1961
Adding party for Thomson1964
Adding party for Thomson1964
Adding party for Thorburn1930
Adding party for Thorby1888
Adding party for Thorp1953
Adding party for Thwaites1980
Adding party for Ticehurst1945
Adding party for Tickner1951
Adding party for Tierney1946
Adding party for Tillem1974
Adding party for Timson1909
Adding party for Tollner1966
Adding party for Toohey1909
Adding party for Townley1907
Adding party for Townley1934
Adding party for Townley1934
Adding party for Townley1934
Adding party for Treloar1892
Adding party for Trenwith1846
Adding party for Trevor1961
Adding party for Troeth1940
Adding party for Trood1948
Adding party for Truss1948
Adding party for Truss1948
Adding party for Tuckey1935
Adding party for Tudge1971
Adding party for Tudor1866
Adding party for Tully1877
Adding party for Turley1859
Adding party for Turnbull1899
Adding party for Turnbull1908
Adding party for Turnbull1908
Adding party for Turnbull190

### adding seats/divisions of politicians